# Image Classification - Lab

## Introduction

Now that you have a working knowledge of CNNs and have practiced implementing associated techniques in Keras, its time to put all of those skills together. In this lab, you'll work to complete a Kaggle competition on classifying dog breeds.

https://www.kaggle.com/c/dog-breed-identification

## Objectives

You will be able to:
* Independently design and build a CNN for image classifcation tasks
* Compare and apply multiple techniques for tuning a model including data augmentation and adapting pretrained models

## Download and Load the Data

Start by downloading the data locally and loading it into a Pandas DataFrame. Be forewarened that this dataset is fairly large and it is advisable to close other memory intensive applications.

The data can be found here:

https://www.kaggle.com/c/dog-breed-identification/data

We recommend downloading the data into this directory on your local computer. From there, be sure to uncompress the folder and subfolders.

In [1]:
#No code persay, but download and decompress the data.

## Preprocessing

Now that you've downloaded the data, its time to prepare it for some model building! You'll notice that the current structure provided is not the same as our lovely preprocessed folders that we've been providing you. Instead, you have one large training folder with images and a csv file with labels associated with each of these file types. 

Use this to create a directory substructure for a train-validation-test split as we have done previously. Also recall from our previous work that you'll also want to use one-hot encoding as we are now presented with a multi-class problem as opposed to simple binary classification.

In [17]:
#Your code here; open the labels.csv file stored in the zip file
import pandas as pd
import numpy as np
import os
import shutil

In [26]:
labels = pd.read_csv('labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [28]:
shutil.move('test.txt', 'data/test.txt')

'data/test.txt'

In [30]:
def train_val_test(X, train_size = 0.75, val_size = 0.15, test_size = 0.15, random_state = None):
    from sklearn.model_selection import train_test_split
    Xtv, test = train_test_split(X, test_size = test_size, shuffle = True, random_state = random_state)
    train, val = train_test_split(Xtv, test_size = val_size, shuffle = True, random_state = random_state)
    
    return train, val, test

In [31]:
breeds = labels.groupby('breed')
train_s, val_s, test_s = 0.70, 0.15, 0.15
for breed, filenames in breeds:
    # Generate train/val/test
    train, val, test = train_val_test(filenames.id)
    files_dict = {'train':train,'val':val,'test':test}
    for folder,IDs in files_dict.items():
        filepath = 'data/'+folder+'/'+breed
        os.mkdir(filepath)
        for ID in IDs:
            src = 'unsorted/'+ID+'.jpg'
            dest = filepath+'/'+ID+'.jpg'
            shutil.move(src, dest)

We wish to create our standard directory structure:
* train
    * category1
    * category2
    * category3
    ...
* val
    * category1
    * category2
    * category3
    ...
* test 
    * category1
    * category2
    * category3
    ...  

In [40]:
#Your code here; transform the image files and then load them into Keras as tensors 
#(be sure to perform a train-val-test split)
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255) 
train_generator = datagen.flow_from_directory(
        'data/train', target_size=(150, 150), 
        batch_size = 10, 
        class_mode ='categorical')
val_generator = datagen.flow_from_directory(
        'data/val', target_size = (150,150),
        batch_size = 10,
        class_mode = 'categorical')
test_generator = datagen.flow_from_directory(
        'data/test', target_size = (150,150),
        batch_size = 10,
        class_mode = 'categorical')

Found 7278 images belonging to 120 classes.
Found 1352 images belonging to 120 classes.
Found 1592 images belonging to 120 classes.


## Optional: Build a Baseline CNN

This is an optional step. Adapting a pretrained model will produce better results, but it may be interesting to create a CNN from scratch as a baseline. If you wish to, do so here.

In [45]:
#Create a baseline CNN model
from keras import models, layers, optimizers
baseline_cnn = models.Sequential()

baseline_cnn.add(layers.Conv2D((32), (3,3), activation = 'relu', input_shape = (150,150,3)))
baseline_cnn.add(layers.MaxPooling2D((2, 2)))

baseline_cnn.add(layers.Flatten())
baseline_cnn.add(layers.Dense(64, activation='relu'))
baseline_cnn.add(layers.Dense(64, activation='relu'))
baseline_cnn.add(layers.Dense(120, activation='softmax'))

baseline_cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

baseline_cnn.fit_generator(train_generator)

Epoch 1/1
728/728 [==============================] - 300s 412ms/step - loss: 4.8644 - acc: 0.0092


## Loading a Pretrained CNN

## Feature Engineering with the Pretrained Model

Now that you've loaded a pretrained model, it's time to adapt that convolutional base and add some fully connected layers on top in order to build a classifier from these feature maps.

In [48]:
#Your code here; add fully connected layers on top of the convolutional base
from keras.applications import VGG19
cnn_VGG19 = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

# set cnn_VGG19 to not trainable
cnn_VGG19.trainable = False

model = models.Sequential()
model.add(cnn_VGG19)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(120, activation = 'softmax'))


model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

model.fit_generator(train_generator, 
                    steps_per_epoch = 15, 
                    epochs = 5, 
                    validation_data = val_generator,
                    validation_steps = 5)

Epoch 1/5
15/15 [==============================] - 417s 28s/step - loss: 4.9009 - acc: 0.0333 - val_loss: 4.9482 - val_acc: 0.0081
Epoch 2/5
15/15 [==============================] - 443s 30s/step - loss: 4.9520 - acc: 0.0067 - val_loss: 4.9007 - val_acc: 0.0081
Epoch 3/5
15/15 [==============================] - 400s 27s/step - loss: 4.8006 - acc: 0.0133 - val_loss: 4.8932 - val_acc: 0.0118
Epoch 4/5
15/15 [==============================] - 425s 28s/step - loss: 4.7963 - acc: 0.0000e+00 - val_loss: 4.7706 - val_acc: 0.0104
Epoch 5/5
15/15 [==============================] - 495s 33s/step - loss: 4.7873 - acc: 0.0067 - val_loss: 4.8100 - val_acc: 0.0111


## Visualize History

Now fit the model and visualize the training and validation accuracy/loss functions over successive epochs.

In [ ]:
#Your code here; visualize the training / validation history associated with fitting the model.

In [ ]:
#Save model

## Final Model Evaluation

In [ ]:
#Your code here

## Summary

Congratulations! In this lab, you brought all of your prior deep learning skills together from preprocessing including one-hot encoding, to adapting a pretrained model. There are always ongoing advancements in CNN architectures and best practices, but you have a solid foundation and understanding at this point.